# SGG YOLO CONVERTER

### This script can convert any SGG annotations file in COCO format (PSG dataset) to YOLO format for object detection training

In [1]:
import os
import shutil
import json
from tqdm import tqdm

data = json.load(open('/home/maelicneau/Documents/ultralytics_relation/psg/psg.json'))

# img_path = '/home/maelic/Documents/Datasets/VG/VG_100K'
# img_data = "./datasets/vg/image_data.json"
# img_data = json.load(open(img_data))

coco_path = "/home/maelic/Documents/Datasets/COCO/"

In [10]:
all_img_path = [data['data'][i]['file_name'] for i in range(len(data['data']))]
dest_folder = "/home/maelic/Documents/Datasets/COCO/PSG_split/"
# move images to new folder
for img in tqdm(all_img_path):
    img_path = os.path.join(coco_path, img)
    dest_path = os.path.join(dest_folder, img)
    # if folder of destination does not exist, create it
    if not os.path.exists(os.path.dirname(dest_path)):
        os.makedirs(os.path.dirname(dest_path))
    shutil.copy(img_path, dest_path)


100%|██████████| 48749/48749 [00:51<00:00, 942.29it/s] 


In [30]:
img_prefix = "/home/maelic/Documents/Datasets/VG/VG_100K"
img_path = img_prefix + '/2370584.jpg'

import cv2
# img size
img = cv2.imread(img_path)
print(img.shape)

(334, 500, 3)


In [2]:
object_to_idx = {}
object_id_to_name = {}
THING_CLASSES = data['thing_classes']
STUFF_CLASSES = data['stuff_classes']
CLASSES = THING_CLASSES + STUFF_CLASSES
for i, cls in enumerate(CLASSES):
    object_to_idx[cls] = str(i)
    object_id_to_name[i] = cls

OUT_PATH = "/home/maelicneau/Documents/DATASETS/PSG"
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)

with open(os.path.join(OUT_PATH, 'classes.txt'), 'w') as f:
    for key,v in object_to_idx.items():
        f.write(key + '\n')

In [7]:
dest_folder = os.path.join(OUT_PATH, 'annotations')
image_dest_folder = os.path.join(OUT_PATH, 'images_all')
from ultralytics.data.utils import visualize_image_annotations

def xyxy_to_xywh(x1, y1, x2, y2, img_width, img_height):
    x_center = (x1 + x2) / 2
    y_center = (y1 + y2) / 2
    w = x2 - x1
    h = y2 - y1
    # Normalize
    x_center /= img_width
    y_center /= img_height
    w /= img_width
    h /= img_height
    return x_center, y_center, w, h

import numpy as np

if not os.path.exists(dest_folder):
    os.makedirs(dest_folder + '/train/labels')
    os.makedirs(dest_folder + '/val/labels')
if not os.path.exists(image_dest_folder):
    os.makedirs(image_dest_folder)

for idx in tqdm(range(len(data['data']))):
    image_data = data['data']
    img_width = image_data[idx]['width']
    img_height = image_data[idx]['height']
    image_id = image_data[idx]['image_id']

    anno = image_data[idx]['annotations']

    if 'train' in image_data[idx]['file_name']:
        out_folder = os.path.join(dest_folder, 'train/labels')
    else:
        out_folder = os.path.join(dest_folder, 'val/labels')

    for obj in anno:
        obj_cat = obj['category_id']
        assert 0 <= obj_cat < 133

        b = obj['bbox']
        # convert to yolo format
        box = xyxy_to_xywh(b[0], b[1], b[2], b[3], img_width, img_height)

        # clip to [0, 1]
        box = np.clip(box, 0, 1)

        with open(os.path.join(out_folder, str(image_id) + '.txt'), 'a') as f:
            f.write(str(obj_cat) + ' ' + str(box[0]) + ' ' + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3])+'\n')
    
    # visualize_image_annotations("/home/maelicneau/Documents/DATASETS/PSG_split/YOLO_anno/train/images/" + str(image_id) + '.jpg', os.path.join(dest_folder, str(image_id) + '.txt'), object_id_to_name)
    # break
    # copy image to dest folder
    # shutil.copyfile(os.path.join(img_path, str(image_id)+'.jpg'), os.path.join(image_dest_folder, str(image_id)+'.jpg'))

100%|██████████| 48749/48749 [00:06<00:00, 7968.61it/s]


### With val split

In [10]:
from sklearn.model_selection import train_test_split

base_path = OUT_PATH
dest_folder = os.path.join(base_path,'annotations')
image_dest_folder = os.path.join(base_path,'images_all')

if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

if not os.path.exists(image_dest_folder):
    os.makedirs(image_dest_folder)

images = [os.path.join(image_dest_folder, x) for x in os.listdir(image_dest_folder)]
annotations = [os.path.join(dest_folder, x) for x in os.listdir(dest_folder) if x[-3:] == "txt"]

images.sort()
annotations.sort()
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.3, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

root_path = 'images/'
folders = [base_path+'/train',base_path+'/test',base_path+'/val']
for folder in folders:
    os.makedirs(os.path.join(folder,root_path))
    
root_path = 'labels/'
for folder in folders:
    os.makedirs(os.path.join(folder,root_path))    

def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

move_files_to_folder(train_images, base_path+'/train/images/')
move_files_to_folder(val_images, base_path+'/val/images/')
move_files_to_folder(test_images, base_path+'/test/images/')
move_files_to_folder(train_annotations, base_path+'/train/labels/')
move_files_to_folder(val_annotations, base_path+'/val/labels/')
move_files_to_folder(test_annotations, base_path+'/test/labels/')

### No val split

In [22]:
from sklearn.model_selection import train_test_split

base_path = OUT_PATH
dest_folder = os.path.join(base_path,'annotations')
image_dest_folder = os.path.join(base_path,'images_all')

if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

if not os.path.exists(image_dest_folder):
    os.makedirs(image_dest_folder)

images = [os.path.join(image_dest_folder, x) for x in os.listdir(image_dest_folder)]
annotations = [os.path.join(dest_folder, x) for x in os.listdir(dest_folder) if x[-3:] == "txt"]

images.sort()
annotations.sort()
train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)

root_path = 'images/'
folders = [base_path+'/train',base_path+'/val']
for folder in folders:
    os.makedirs(os.path.join(folder,root_path))
    
root_path = 'labels/'
for folder in folders:
    os.makedirs(os.path.join(folder,root_path))    

def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

move_files_to_folder(train_images, base_path+'/train/images/')
move_files_to_folder(test_images, base_path+'/val/images/')
move_files_to_folder(train_annotations, base_path+'/train/labels/')
move_files_to_folder(test_annotations, base_path+'/val/labels/')

In [11]:
# write the yaml file
classes = object_to_idx.keys()

train_path = base_path+'train/'
val_path = base_path+'val/'
test_path = base_path+'test/'

n_classes = len(classes)

with open(base_path+'/train.yaml', 'w') as f:
    f.write('train: '+train_path+'\n')
    f.write('val: '+val_path+'\n')
    f.write('test: '+test_path+'\n')
    f.write('nc: '+str(n_classes)+'\n')
    f.write('names: '+str(list(classes)))